In [110]:
import importlib
import functions as fn
importlib.reload(fn)

<module 'functions' from 'c:\\Users\\user1\\Documents\\24\\functions.py'>

In [73]:
conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()


In [74]:
symbol = 'BANKNIFTY'  # Replace with your symbol
expiry_future = fn.fetch_expiries(cursor, symbol)


In [75]:

for i, row in expiry_future.items():
    print(i, row, sep=' = ', end=' | ')
    if (i + 1) % 6 == 0:
        print()

0 = 2022-05-26 | 1 = 2022-06-30 | 2 = 2022-07-28 | 3 = 2022-08-25 | 4 = 2022-09-29 | 5 = 2022-10-27 | 
6 = 2022-11-24 | 7 = 2022-12-29 | 8 = 2023-01-25 | 9 = 2023-02-23 | 10 = 2023-03-29 | 11 = 2023-04-27 | 
12 = 2023-05-25 | 13 = 2023-06-28 | 14 = 2023-06-29 | 15 = 2023-07-27 | 16 = 2023-08-31 | 17 = 2023-09-28 | 
18 = 2023-10-26 | 19 = 2023-11-30 | 20 = 2023-12-28 | 21 = 2024-01-25 | 22 = 2024-02-29 | 23 = 2024-03-27 | 
24 = 2024-04-24 | 25 = 2024-05-29 | 26 = 2024-06-26 | 27 = 2024-07-31 | 

In [76]:
index = 20

# Fetch the futures data for the selected expiry date
futures = fn.fetch_futures(cursor, symbol,expiry_future[index])
# Convert the 'date_timestamp' column to datetime if it exists
futures.drop(columns=['id'] , inplace=True) ; futures.drop_duplicates(inplace=True)


In [79]:
fn.fill_missings(futures)

In [80]:
fn.genrate_signals(futures)

In [81]:
options = fn.fetch_options(cursor, symbol=symbol, expiry=str(expiry_future[index]))

In [82]:
calls , puts = fn.fetch_call_put(options)

In [103]:
trades = fn.generate_trades(futures,calls,puts)

In [104]:
fig = fn.grpah(x=[i for i in range(len(trades))] , y=[i['pnl'] for i in trades])


In [105]:
print(trades)

[{'pos': 0, 'signal_time': Timestamp('2023-12-01 09:16:00'), 'long_ema': np.float64(4482331.296296297), 'short_ema': np.float64(4482325.0), 'entry_time': Timestamp('2023-12-01 09:17:00'), 'strike': np.int64(4500000), 'entry_price': np.float64(67500.0), 'exit_time': Timestamp('2023-12-01 09:18:00'), 'exit_price': np.float64(67000.0), 'pnl': np.float64(-500.0), 'type': 'PE', 'pnl_sum': np.float64(-500.0), 'price': np.int64(4482285)}, {'pos': 0, 'signal_time': Timestamp('2023-12-01 09:17:00'), 'long_ema': np.float64(4482395.644718793), 'short_ema': np.float64(4482500.0), 'entry_time': Timestamp('2023-12-01 09:18:00'), 'strike': np.int64(4480000), 'entry_price': np.float64(62300.0), 'exit_time': Timestamp('2023-12-01 10:38:00'), 'exit_price': np.float64(79750.0), 'pnl': np.float64(17450.0), 'type': 'CE', 'pnl_sum': np.float64(16950.0), 'price': np.int64(4483200)}, {'pos': 0, 'signal_time': Timestamp('2023-12-01 10:37:00'), 'long_ema': np.float64(4512325.156835257), 'short_ema': np.float64(

In [106]:
fig = fn.grpah(x=[i for i in range(len(trades))] , y=[i['pnl_sum'] for i in trades])

In [107]:
drawdownval = fn.drawdown(trades)
print(drawdownval)

173600.0


In [108]:
for i in trades:
    print(f" signal time {i['signal_time']} | price {i['price']} |  on date_time {i['entry_time']} | strike {i['strike']} | type : {i['type']}|  buy price {i['entry_price']}  |   sold at {i['exit_price']} on {i['exit_time']} , short_ema {i['short_ema']} | long_ema {i['long_ema']} |  PnL = {i['pnl']}")

 signal time 2023-12-01 09:16:00 | price 4482285 |  on date_time 2023-12-01 09:17:00 | strike 4500000 | type : PE|  buy price 67500.0  |   sold at 67000.0 on 2023-12-01 09:18:00 , short_ema 4482325.0 | long_ema 4482331.296296297 |  PnL = -500.0
 signal time 2023-12-01 09:17:00 | price 4483200 |  on date_time 2023-12-01 09:18:00 | strike 4480000 | type : CE|  buy price 62300.0  |   sold at 79750.0 on 2023-12-01 10:38:00 , short_ema 4482500.0 | long_ema 4482395.644718793 |  PnL = 17450.0
 signal time 2023-12-01 10:37:00 | price 4511600 |  on date_time 2023-12-01 10:38:00 | strike 4530000 | type : PE|  buy price 10700.0  |   sold at 67560.0 on 2023-12-01 10:39:00 , short_ema 4512317.106309336 | long_ema 4512325.156835257 |  PnL = 56860.0
 signal time 2023-12-01 10:38:00 | price 4512700 |  on date_time 2023-12-01 10:39:00 | strike 4510000 | type : CE|  buy price 62850.0  |   sold at 61080.0 on 2023-12-01 10:43:00 , short_ema 4512393.685047468 | long_ema 4512352.922995608 |  PnL = -1770.0
 

In [111]:
print(f"sharpie ratio = {fn.sharpie(trades)}")

sharpie ratio = 0.11523192994542368
